In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import binom
import math
import inspect # print(inspect.getsource(MyFunction)) shows the source code for MyFunction
import source_ACR_py as acr

# Initialization
num_S = 2
max_order = 2
num_R = 2
num_total_C = int(binom(max_order + num_S, num_S))
total_complex = acr.make_total_complexes(num_S, max_order)

num_total_R = num_total_C * (num_total_C - 1)

path = "/Users/hyukpyohong/Dropbox/CRN_dynamicACR/Codes/data/"

# Load data
# Warning: matrices data might be loaded as their transposes.
list_acr_id = pd.read_csv(path + f"python_list_acr_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=None).transpose().to_numpy()
list_unbnd_id = pd.read_csv(path + f"python_list_unbnd_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=None).transpose().to_numpy()
matrix_R_id = pd.read_csv(path + f"python_matrix_R_id_S{num_S}R{num_R}_max_ord{max_order}.csv", header=None).transpose().to_numpy()

net_list_with_acr = np.where(list_acr_id[0, :] > 0)[0]
num_total_net = list_acr_id.shape[1]

In [3]:
plot_array = [None] * num_total_net
group_list = np.zeros(num_total_net, dtype='<U3')

for net_id in range(1, num_total_net + 1):
    if net_id % 50 == 1:
        print(f"======= Grouping network {net_id:4} =======")
    
    list_R_id = matrix_R_id[:, net_id - 1]
    list_source_id = np.zeros(num_R, dtype=int)
    list_product_id = np.zeros(num_R, dtype=int)

    for i in range(num_R):
        s_id, p_id = acr.reaction_to_complex(list_R_id[i], num_total_C)
        list_source_id[i] = s_id
        list_product_id[i] = p_id

    source_mat = total_complex[:, list_source_id-1]
    product_mat = total_complex[:, list_product_id-1]
    
    #network_txt, network_txt_short = acr.crn_writing(source_mat, product_mat)
    acr_id = list_acr_id[:, net_id - 1]
    unbnd_id = list_unbnd_id[:, net_id - 1]
    group_list[net_id - 1] = acr.grouping_network(source_mat, product_mat, acr_id, unbnd_id)


======= Grouping network    1 =======
======= Grouping network   51 =======
======= Grouping network  101 =======
======= Grouping network  151 =======
======= Grouping network  201 =======
======= Grouping network  251 =======
======= Grouping network  301 =======
======= Grouping network  351 =======
======= Grouping network  401 =======


In [4]:
group_size_dict = {}
for r in [1,2]:
    for id_acr in [0,1,2,3]:
        for id_unbnd in [0,1,2,3]:
            group_size_dict[str(r)+str(id_acr)+str(id_unbnd)] = 0

path = "/Users/hyukpyohong/Dropbox/CRN_dynamicACR/Codes/plots/"
layout1 = [3,4] # the layout for plotting CRNs. (3,4) means three rows and 4 columns.
num_grid = np.prod(layout1)

for gp in group_list:
    group_size_dict[gp] += 1

for gp in group_size_dict.keys():
    gp_size = group_size_dict[gp]
    num_sheets = math.ceil(gp_size/num_grid)
    net_id_list_in_gp = np.where(group_list == gp)[0] + 1 # We add 1 to make net_id start from 1 not 0.
    
    for i in range(len(net_id_list_in_gp)):
        page = math.floor(i/num_grid)+1
        if (i % num_grid == 0):
            if page % 5 == 1:
                print(f"======= Drawing page {page:4} for group {gp:3} =======")
            
            if num_S == 2:
                fig, axes = plt.subplots(layout1[0], layout1[1], figsize=(12, 9))
            elif num_S == 3:
                fig, axes = plt.subplots(layout1[0], layout1[1], figsize=(12, 9), subplot_kw=dict(projection='3d'))
            else:
                print('The number of species must be either 2 or 3 to draw plots.')
                break

        net_id = net_id_list_in_gp[i]
        list_R_id = matrix_R_id[:, net_id - 1]
        list_source_id = np.zeros(num_R, dtype=int)
        list_product_id = np.zeros(num_R, dtype=int)

        for j in range(num_R):
            s_id, p_id = acr.reaction_to_complex(list_R_id[j], num_total_C)
            list_source_id[j] = s_id
            list_product_id[j] = p_id
            
        source_mat = total_complex[:, list_source_id-1]
        product_mat = total_complex[:, list_product_id-1]
        #network_txt, network_txt_short = acr.crn_writing(source_mat, product_mat)
        acr_id = list_acr_id[:, net_id - 1]
        unbnd_id = list_unbnd_id[:, net_id - 1]
        rr = math.floor((i%num_grid)/layout1[1])
        cc = i%layout1[1]
        
        acr.add_CRN_to_Axes(axes, rr, cc, source_mat, product_mat, acr_id, unbnd_id)
        
        if (i % num_grid == num_grid-1 or i == len(net_id_list_in_gp)-1):
            if max(group_size_dict.values()) < 10:
                plt.savefig(f"{path}Grouped_S{num_S}R{num_R}_max_ord{max_order}_gp{gp}_{page:01}.png")
            elif max(group_size_dict.values()) < 100:
                plt.savefig(f"{path}Grouped_S{num_S}R{num_R}_max_ord{max_order}_gp{gp}_{page:02}.png")
            elif max(group_size_dict.values()) < 1000:
                plt.savefig(f"{path}Grouped_S{num_S}R{num_R}_max_ord{max_order}_gp{gp}_{page:03}.png")
            else:
                plt.savefig(f"{path}Grouped_S{num_S}R{num_R}_max_ord{max_order}_gp{gp}_{page:04}.png")
            plt.close() # Maybe, we need to move it above.
            

======= Drawing page    1 for group 100 =======
======= Drawing page    1 for group 101 =======
======= Drawing page    1 for group 102 =======
======= Drawing page    1 for group 110 =======
======= Drawing page    1 for group 120 =======
======= Drawing page    1 for group 200 =======
======= Drawing page    6 for group 200 =======
======= Drawing page    1 for group 201 =======
======= Drawing page    1 for group 202 =======
======= Drawing page    1 for group 203 =======
======= Drawing page    1 for group 210 =======
======= Drawing page    1 for group 212 =======
======= Drawing page    1 for group 220 =======
======= Drawing page    1 for group 221 =======
======= Drawing page    1 for group 223 =======
======= Drawing page    1 for group 230 =======
======= Drawing page    1 for group 231 =======
======= Drawing page    1 for group 232 =======
